# Python API examples


## Creates

- createUser
- createMarket
- createTrade

### Create user

Creates a new user. Returns signature and verify keys.

~~~~
import requests
url = 'https://blocmarket.herokuapp.com/createUser'
headers = {'content-type': 'application/json'}
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)
~~~~

Example return as json:

~~~~
{'signingKey': '0cca0a3739eba76cc78823d9f6e898379014d8c53172f5e45c171a3c54a9f477',
 'traderId': '1',
 'verifyKey': 'cdcfb59431b2579a681cee65656cbed6f8d578d5cc30d3e759e86c1d3e3529ef'}
~~~~
        
### Create/update market

Creates/updates a market. Returns True and market details if successful. 

Specify:
- Market root and branch id
- Minimum and maximum outcomes
- Trader Id
- Sign and verify keys

Note that sending the private key is a terrible idea. In production signing is done on client side.

~~~~
sk = '0cca0a3739eba76cc78823d9f6e898379014d8c53172f5e45c171a3c54a9f477'
vk = 'cdcfb59431b2579a681cee65656cbed6f8d578d5cc30d3e759e86c1d3e3529ef'
tId = 2

url = 'https://blocmarket.herokuapp.com/createMarket'
headers = {'content-type': 'application/json'}
content_makemarket = {"signingKey": sk,
                    "traderId": tId, 
                    "verifyKey": vk,
                    "marketRootId": 1, 
                    "marketBranchId": 1, 
                    "marketMin": 0,
                    "marketMax":1e4}

response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
~~~~

Example return as json:

~~~~
{'allChecks': "{'inputChk': True, 'marketLimitChk': True, 'traderIdChk': True, 'marketId': '1', 'marketRangeChk': True, 'marketIndChk': True, 'sigChk': True, 'chainChk': True, 'ownerChk': False, 'timeChk': True}",
 'checks': 'True',
 'marketBranchId': 1,
 'marketId': 1,
 'marketMax': 1e4,
 'marketMin': 0,
 'marketRootId': 1,
 'traderId': 2}

~~~~
    
### Create trade

Create a trade. Returns True and trade details if successful.

Specify:

- Trader Id
- Market Id
- Price
- Quantity
- Sign and verify keys

~~~~
url = 'https://blocmarket.herokuapp.com/createTrade'
headers = {'content-type': 'application/json'}
content_maketrade = {"signingKey": sk,
                     "traderId": int(tId),
                     "verifyKey": vk,
                     "marketId": mkId,
                     "price": 5510,
                     "quantity":1}
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)



~~~~

Example return as json:
~~~~

{'allChecks': "{'inputChk': True, 'traderIdChk': True, 'marketChk': True, 'sigChk': True, 'chainChk': True, 'timeChk': True, 'colChk': True}",
 'checks': 'True',
 'marketId': 1,
 'price': 5510,
 'quantity': 1,
 'traderId': 1}


~~~~

## Views

- viewMarketBounds
- viewOrderBook
- viewOpenTrades
- viewMatchedTrades

### View markets

~~~~
url = 'https://blocmarket.herokuapp.com/viewMarketBounds'
headers = {'content-type': 'application/json'}
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)
~~~~

Example response as json:

~~~~
'{"marketId":{"0":1,"1":2,"2":3,"3":4},
"marketRootId":{"0":1,"1":2,"2":3,"3":4},
"marketBranchId":{"0":1,"1":1,"2":1,"3":1},
"marketMin":{"0":0.0,"1":0.0,"2":0.0,"3":0.0},
"marketMax":{"0":,"1":7000,"2":7000,"3":7000}}'
~~~~

### View full order book

~~~~
url = 'https://blocmarket.herokuapp.com/viewOrderBook'
headers = {'content-type': 'application/json'}
content = {'marketId': 1}
response = requests.post(url, data=json.dumps(content), headers=headers, stream=True)
~~~~

Example response as json:

~~~~
'{"marketId":{"0":1,"1":1},"price":{"0":5510,"1":5510},
"quantity":{"0":1.0,"1":-1.0,"traderId":{"0":1,"1":1},
"iMatched":{"0":true,"1":true,
"timeStampUTC":{"0":1547976585838,"1":1547976602766}}'
~~~~

Using a stream is only necessary if the order book is large and the request is truncated by the server, but it doesn't do any harm.

### View open trades

~~~~
url = 'https://https://blocmarket.herokuapp.com/viewOpenTrades'
content = {'marketId': 1}
response = requests.post(url, data=json.dumps(content), headers=headers, stream=True)
~~~~

~~~~
'{"marketId":{"12":1,"15":1,"18":1,"21":1,"24":1},
"price":{"12":4300,"15":4300,"18":4300,"21":4300,"24":4300},
"quantity":{"12":-1.0,"15":-1.0,"18":-1.0,"21":-1.0,"24":-1.0},
"traderId":{"12":1,"15":1,"18":1,"21":1,"24":1},"timeStampUTC":
{"12":1547976742476,"15":1547976750344,"18":1547976758683,"21":1547976768968,"24":1547976775537}}'
~~~~

### View matched trades

~~~~
url = 'https://blocmarket.herokuapp.com/viewMatchedTrades'
headers = {'content-type': 'application/json'}
content = {'marketId': 1}
response = requests.post(url, data=json.dumps(content), headers=headers, stream=True)
~~~~

Example response as json:

~~~~
'{"marketId":{"0":1,"1":1},"price":{"0":4300,"1":5500},"quantity":{"0":0.0,"1":0.0},"traderId":{"0":1,"1":1}}'

~~~~

### View trader summary

A view joining order book and marketTable for a particular trader.

~~~~
url = 'https://blocmarket.herokuapp.com/viewTradeSummary'
headers = {'content-type': 'application/json'}
content = {'traderId': 2}
response = requests.post(url, data=json.dumps(content), headers=headers, stream=True)
~~~~

Example response as json:

~~~~
'{"marketId":{"0":1,"1":1},"price":{"0":8000,"1":5000},
"quantity":{"0":-1.0,"1":-1.0},"traderId":{"0":2,"1":2},
"iMatched":{"0":false,"1":true},
"timeStampUTC":{"0":1547982939316,"1":1547982938964},
"marketMin":{"0":0.0,"1":0.0},"marketMax":{"0":1e4,"1":1e4}}'
~~~~


# Python demonstration

In [1]:
import requests
import json
import numpy as np
import pandas as pd
import time
import datetime

%matplotlib inline
import matplotlib.pyplot as plt

# Running flask app from powershell
# $env:FLASK_APP = "api"
# $env:FLASK_ENV = "development"
# flask run

# Running flask from terminal on OSX
# export FLASK_APP=api
# export FLASK_ENV=development
# flask run

# Running with gunicorn (heroku, digitalocean, etc):
# Procfile >
# web: gunicorn wsgi --log-file -
# wsgi.py >
# from api import app as application
#
#if __name__ == "__main__":
#    application.run()

# https://devcenter.heroku.com/articles/getting-started-with-python
# https://www.digitalocean.com/community/tutorials/how-to-serve-flask-applications-with-gunicorn-and-nginx-on-ubuntu-14-04


In [19]:
baseurl = 'https://blocmarket.herokuapp.com/'
#baseurl = 'http://127.0.0.1:7000/'
baseurl = 'http://127.0.0.1:5000/'

In [3]:
# Create a new user
url = baseurl +'createUser'
headers = {'content-type': 'application/json'}
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [4]:
# Copy keys and traderid from response
sk = response.json()['signingKey']
vk = response.json()['verifyKey']
tId = int(response.json()['traderId'])


In [5]:
sk, tId, vk, mkRtId, mkBrId, 1500, 20000, json.dumps({'field': 1})

NameError: name 'mkRtId' is not defined

In [70]:
# Create a new market
mkRtId = 5
mkBrId = 1


url = baseurl + 'createMarket'
content_makemarket = {"signingKey": sk,
                        "traderId": tId, 
                        "verifyKey": vk,
                        "marketRootId": mkRtId, 
                        "marketBranchId": mkBrId, 
                        "marketMin": 1100,
                        "marketMax":7700,
                        "marketDesc": json.dumps({})}
# Post market
response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)


In [122]:
# Create a new market
mkRtId = 5
mkBrId = 1


url = baseurl + 'createMarket'
content_makemarket = {"signingKey": sk,
                        "traderId": tId, 
                        "verifyKey": vk,
                        "marketRootId": 1, 
                        "marketBranchId": 30, 
                        "marketMin": 1100,
                        "marketMax":7700,
                        "marketDesc": {'abc'}
# Post market
response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)


In [123]:
response.json()

{'allChecks': "{'Boned': True, 'marketId': 0}",
 'checks': 'Traceback (most recent call last):\n  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sqlalchemy/engine/base.py", line 1245, in _execute_context\n    self.dialect.do_execute(\n  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sqlalchemy/engine/default.py", line 581, in do_execute\n    cursor.execute(statement, parameters)\npsycopg2.errors.InvalidTextRepresentation: invalid input syntax for type json\nLINE 1: ...dde2d2e908648102a34be796851907de82d6e600\'::bytea, \'Market ro...\n                                                             ^\nDETAIL:  Token "Market" is invalid.\nCONTEXT:  JSON data, line 1: Market...\n\n\nThe above exception was the direct cause of the following exception:\n\nTraceback (most recent call last):\n  File "/Users/josephclark/src/blocmarket/BlocAPI.py", line 220, in createMarket\n    checks, allChecks = bc.createMarket_client(mark

In [71]:
# response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
# View response as json
response.json()

{'allChecks': "{'inputChk': True, 'marketLimitChk': True, 'traderIdChk': True, 'marketId': '5', 'marketRangeChk': True, 'marketIndChk': True, 'sigChk': True, 'chainChk': True, 'ownerChk': False, 'timeChk': True}",
 'checks': 'False',
 'marketBranchId': 1,
 'marketId': 5,
 'marketMax': 7700,
 'marketMin': 1100,
 'marketRootId': 8,
 'traderId': 8}

In [72]:
# Save market id from response
mkId = response.json()['marketId']
mkId

5

In [128]:
# Create a trade
url = baseurl+'createTrade'

content_maketrade = {"signingKey": sk,
                     "traderId": tId,
                     "verifyKey": vk,
                     "marketId": mkId,
                     "price": 5510,
                     "quantity":1}
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [129]:
# View response as json
response.json()

{'allChecks': "{'tradeId': 164, 'inputChk': True, 'traderIdChk': True, 'marketChk': True, 'sigChk': True, 'chainChk': True, 'timeChk': True, 'colChk': True}",
 'checks': 'True',
 'marketId': 5,
 'price': 5510,
 'quantity': 1,
 'tradeId': 164,
 'traderId': 19}

In [75]:
# View response as a dataframe
pd.DataFrame(response.json(), index=[0])


,allChecks,checks,marketId,price,quantity,tradeId,traderId
0,"{'tradeId': 130, 'inputChk': True, 'traderIdCh...",True,5,5510,1,130,8


In [76]:
mId = response.json()['marketId']

In [77]:
# Create a matched trade
url = baseurl+'createTrade'
content_maketrade['quantity'] = -1
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [78]:
# View
response.json()

{'allChecks': "{'tradeId': 131, 'inputChk': True, 'traderIdChk': True, 'marketChk': True, 'sigChk': True, 'chainChk': True, 'timeChk': True, 'colChk': True}",
 'checks': 'True',
 'marketId': 5,
 'price': 5510,
 'quantity': -1,
 'tradeId': 131,
 'traderId': 8}

In [79]:
# Create many matched trades
url = baseurl+'createTrade'

for i in range(5):
    t = time.time()
    content_maketrade['quantity'] = -1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    content_maketrade['quantity']=1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 0.42373204231262207 seconds.
matched trade: 1 in 0.3921778202056885 seconds.
matched trade: 2 in 0.39849328994750977 seconds.
matched trade: 3 in 0.41433000564575195 seconds.
matched trade: 4 in 0.4284541606903076 seconds.


In [80]:
content_maketrade

{'signingKey': 'e3564e21b200e569484c13a4ad06693e7da4e764d7c644dfae421bf6f11e4067',
 'traderId': 8,
 'verifyKey': '733042e9a8364a2b68396ad053347476a74b9201b698c72a548282be1ae68036',
 'marketId': 5,
 'price': 5510,
 'quantity': 1}

In [81]:
# Create many markets
url = baseurl+'createMarket'

for i in range(11):
    t = time.time()

    content_makemarket = {"signingKey": sk,
      "traderId": int(tId), "verifyKey": vk,
      "marketRootId": i+1, "marketBranchId": 1,
      "marketMin": 0, "marketMax":70, "marketDesc": json.dumps({})}

    response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
    if response.json()['checks'] == 'True':
        print('market created: ' + str(i+1) + ' in ' + str(time.time() - t) + ' seconds.')
    else:
        print('failed to create market ' +str(i+1))
    

failed to create market 1
failed to create market 2
failed to create market 3
failed to create market 4
failed to create market 5
failed to create market 6
failed to create market 7
failed to create market 8
failed to create market 9
failed to create market 10
failed to create market 11


In [82]:
response.json()

{'allChecks': "{'inputChk': True, 'marketLimitChk': True, 'traderIdChk': True, 'marketId': '12', 'marketRangeChk': True, 'marketIndChk': True, 'sigChk': True, 'chainChk': True, 'ownerChk': False, 'timeChk': True}",
 'checks': 'False',
 'marketBranchId': 1,
 'marketId': 12,
 'marketMax': 70,
 'marketMin': 0,
 'marketRootId': 11,
 'traderId': 8}

In [83]:
# Create many sub-markets
url = baseurl+'createMarket'
marketRootId = mId

for i in range(10):
    t = time.time()

    content_makemarket = {"signingKey": sk,
      "traderId": int(tId), "verifyKey": vk,
      "marketRootId": marketRootId, "marketBranchId": 1+i, "marketMin": 0, "marketMax":70, "marketDesc": json.dumps({})}

    response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
    if response.json()['checks'] == 'True':
        print('market created: ' + str(i+1) + ' in ' + str(time.time() - t) + ' seconds.')
    else:
        print('failed to create sub market ' +str(i+1))
    

failed to create sub market 1
failed to create sub market 2
failed to create sub market 3
failed to create sub market 4
failed to create sub market 5
failed to create sub market 6
failed to create sub market 7
failed to create sub market 8
failed to create sub market 9
failed to create sub market 10


In [84]:
response.json()

{'allChecks': "{'inputChk': True, 'marketLimitChk': True, 'traderIdChk': True, 'marketId': '21', 'marketRangeChk': True, 'marketIndChk': True, 'sigChk': True, 'chainChk': True, 'ownerChk': False, 'timeChk': True}",
 'checks': 'False',
 'marketBranchId': 10,
 'marketId': 21,
 'marketMax': 70,
 'marketMin': 0,
 'marketRootId': 5,
 'traderId': 8}

In [20]:
# View maximum and minimum of each market
url = baseurl+'viewMarketBounds'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)
mB = pd.read_json(response.json())
mB

,index,marketId,marketRootId,marketBranchId,marketMin,marketMax,marketDesc
0,0,4,5,1,1100,7700,{}
1,1,5,8,1,1100,1100,{}
10,10,28,66,1,1000,1111,{}
11,11,29,65,1,500,600,{}
12,12,6,3,1,0,70,{}
13,13,9,7,1,0,70,{}
14,14,11,10,1,0,70,{}
15,15,14,5,3,1100,1100,{}
16,16,17,5,6,1100,1100,{}
17,17,20,5,9,1100,1100,{}


In [33]:
mBmin = mB.loc[mB.marketId==1, 'marketMin'].values[0]
mBmin


0

In [86]:
# View whole order book
url = baseurl+'viewOrderBook'
content = {'marketId': mId}
response = requests.post(url, data=json.dumps(content), headers=headers, stream=True)

In [87]:
pd.read_json(response.json())


,index,tradeId,marketId,price,quantity,traderId,iMatched,timeStampUTC
0,13,119,5,5510,-1,6,True,2020-01-04 11:22:27.482
1,14,118,5,5510,1,6,True,2020-01-04 11:22:21.980
10,23,128,5,5510,-1,6,True,2020-01-04 11:22:34.248
11,24,129,5,5510,1,6,True,2020-01-04 11:22:34.433
12,25,131,5,5510,-1,8,True,2020-01-05 09:09:09.326
13,26,130,5,5510,1,8,True,2020-01-05 09:08:58.243
14,27,132,5,5510,-1,8,True,2020-01-05 09:09:13.651
15,28,133,5,5510,1,8,True,2020-01-05 09:09:13.853
16,29,134,5,5510,-1,8,True,2020-01-05 09:09:14.047
17,30,135,5,5510,1,8,True,2020-01-05 09:09:14.245


In [88]:
jsdata = json.loads(response.json())
jsdata['tradeId']

{'0': 119,
 '1': 118,
 '2': 120,
 '3': 121,
 '4': 122,
 '5': 123,
 '6': 124,
 '7': 125,
 '8': 126,
 '9': 127,
 '10': 128,
 '11': 129,
 '12': 131,
 '13': 130,
 '14': 132,
 '15': 133,
 '16': 134,
 '17': 135,
 '18': 136,
 '19': 137,
 '20': 138,
 '21': 139,
 '22': 140,
 '23': 141}

In [89]:
content_makemarket

{'signingKey': 'e3564e21b200e569484c13a4ad06693e7da4e764d7c644dfae421bf6f11e4067',
 'traderId': 8,
 'verifyKey': '733042e9a8364a2b68396ad053347476a74b9201b698c72a548282be1ae68036',
 'marketRootId': 5,
 'marketBranchId': 10,
 'marketMin': 0,
 'marketMax': 70,
 'marketDesc': '{}'}

In [90]:
# Create many unmatched trades
url = baseurl+'createTrade'

for i in range(5):
    t = time.time()
    content_maketrade['quantity'] = -1
    content_maketrade['price'] = 4300
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('unmatched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


unmatched trade: 0 in 0.24561786651611328 seconds.
unmatched trade: 1 in 0.19882512092590332 seconds.
unmatched trade: 2 in 0.1965961456298828 seconds.
unmatched trade: 3 in 0.19463086128234863 seconds.
unmatched trade: 4 in 0.19795703887939453 seconds.


In [91]:
a = response.json()
a

{'allChecks': "{'tradeId': 146, 'inputChk': True, 'traderIdChk': True, 'marketChk': True, 'sigChk': True, 'chainChk': True, 'timeChk': True, 'colChk': True}",
 'checks': 'True',
 'marketId': 5,
 'price': 4300,
 'quantity': -1,
 'tradeId': 146,
 'traderId': 8}

In [92]:
# View all open (unmatched trades)
url = baseurl+'viewOpenTrades'

content = {'marketId': mId}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [93]:
pd.read_json(response.json())

,index,tradeId,marketId,price,quantity,traderId,iMatched,timeStampUTC
0,36,141,5,5510,1,8,False,2020-01-05 09:09:15.486
1,37,142,5,4300,-1,8,False,2020-01-05 09:09:53.647
2,38,143,5,4300,-1,8,False,2020-01-05 09:09:53.856
3,39,144,5,4300,-1,8,False,2020-01-05 09:09:54.050
4,40,145,5,4300,-1,8,False,2020-01-05 09:09:54.244
5,41,146,5,4300,-1,8,False,2020-01-05 09:09:54.442


In [94]:
# View matched trades (summed over marketId/price/traderId)
url = baseurl+'viewMatchedTrades'
content = {'marketId': mkId}
response = requests.post(url, data=json.dumps(content), headers=headers, stream=True)

In [95]:
pd.read_json(response.json())

,index,marketId,price,quantity,traderId


In [96]:
# Create many matched trades (own)
url = baseurl+'createTrade'

for i in range(5): 
    t = time.time()
    content_maketrade['quantity'] = -1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    content_maketrade['quantity']=1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 0.42885708808898926 seconds.
matched trade: 1 in 0.39078402519226074 seconds.
matched trade: 2 in 0.39959192276000977 seconds.
matched trade: 3 in 0.3867061138153076 seconds.
matched trade: 4 in 0.3878011703491211 seconds.


In [97]:
# View summary of trader for a particular trader
url = baseurl+'viewTradeSummary'
content = {'traderId': tId}
response = requests.post(url, data=json.dumps(content), headers=headers, stream=True)

In [98]:
pd.read_json(response.json())

,index,tradeId,marketId,price,quantity,traderId,iMatched,timeStampUTC,marketRootId,marketBranchId,marketMin,marketMax,marketMinOutcome,marketMaxOutcome
0,0,131,5,5510,-1,8,True,2020-01-05 09:09:09.326,8,1,1100,1100,4410,4410
1,1,130,5,5510,1,8,True,2020-01-05 09:08:58.243,8,1,1100,1100,-4410,-4410
10,10,140,5,5510,-1,8,True,2020-01-05 09:09:15.268,8,1,1100,1100,4410,4410
11,11,141,5,5510,1,8,False,2020-01-05 09:09:15.486,8,1,1100,1100,-4410,-4410
12,12,147,5,4300,-1,8,False,2020-01-05 09:10:10.484,8,1,1100,1100,3200,3200
13,13,142,5,4300,-1,8,True,2020-01-05 09:09:53.647,8,1,1100,1100,3200,3200
14,14,148,5,4300,1,8,True,2020-01-05 09:10:10.687,8,1,1100,1100,-3200,-3200
15,15,149,5,4300,-1,8,False,2020-01-05 09:10:10.882,8,1,1100,1100,3200,3200
16,16,143,5,4300,-1,8,True,2020-01-05 09:09:53.856,8,1,1100,1100,3200,3200
17,17,150,5,4300,1,8,True,2020-01-05 09:10:11.077,8,1,1100,1100,-3200,-3200


In [99]:
# Get a signed UTC timestamp
url = baseurl+'getSignedUTCTimestamp'
content = {}
response = requests.get(url, data=json.dumps(content), headers=headers)


In [100]:
response.json()

{'timeStampUTC': '2020-01-05 09:10:22.752896',
 'timeStampUTCSignature': 'b"\\xe4V-\\x81\\x11{\\xaf\\xa3C\\xde\\xc7\\r\\x18Coh\\xd1\\xe6v\\x1a:\\xf0\\xc6\\xe8&\\x89=\\xf7\\xa3?l\\x9bp\\n\\x07\\x1ac\\x8e\\xf4y\\xde[\\x9b\\x13\\x15\'\\x8a\\xa1\\xa0\\xcbs{\\xa7~\\x83\\x96\\x8b\\x85P\\xfa\\xf9*\\xea\\t"',
 'verifyKey': 'cdcfb59431b2579a681cee65656cbed6f8d578d5cc30d3e759e86c1d3e3529ef'}

In [101]:
# View summary of trader for a particular trader
url = baseurl+'viewTickHistory'
content = {'marketId': mId, 'startTime': datetime.datetime(2019,1,1).timestamp()*1000, 'endTime': 2e9*1000}
response = requests.post(url, data=json.dumps(content), headers=headers, stream=True)

In [102]:
pd.read_json(response.json())

,index,tickType,tradeId,xTradeId,marketId,price,quantity,traderId,iMatched,timeStampUTC
0,0,BID,118,119.0,5,5510,1,6,True,2020-01-04 11:22:21.980
1,1,TRADE,129,131.0,5,5510,1,6,True,2020-01-04 11:22:34.433
2,2,ASK,142,148.0,5,4300,-1,8,True,2020-01-05 09:09:53.647
3,3,ASK,143,148.0,5,4300,-1,8,True,2020-01-05 09:09:53.856
4,4,ASK,144,148.0,5,4300,-1,8,True,2020-01-05 09:09:54.050
5,5,ASK,145,148.0,5,4300,-1,8,True,2020-01-05 09:09:54.244
6,6,ASK,146,148.0,5,4300,-1,8,True,2020-01-05 09:09:54.442
7,7,ASK,147,NaN,5,4300,-1,8,False,2020-01-05 09:10:10.484



# Curl API examples



## Create user

~~~~
curl --header "Content-Type: application/json" -X POST http://blocmarket.herokuapp.com/createUser
~~~~
        
## Create market

~~~~
curl --data '{"signingKey_hex": "ece2efc138c8298d43caba1315ceda614e20644c74d46fed37871c47ea19afdf",
"traderId": 1, 
"verifyKey_hex": "9690a2e12971ae452d68bf3d08405090d45791533cf80740fd186aea4b6773fc",
"marketRootId": 1, 
"marketBranchId": 5, 
"marketMin": 0, 
"marketMax": 1e4}' 
--header "Content-Type: application/json" -X POST http://blocmarket.herokuapp.com/createMarket
~~~~
    
## Create trade

~~~~
curl --data '{"signingKey_hex": "ece2efc138c8298d43caba1315ceda614e20644c74d46fed37871c47ea19afdf",
"traderId": 1, 
"verifyKey_hex": "9690a2e12971ae452d68bf3d08405090d45791533cf80740fd186aea4b6773fc",
"marketId": 1,
"price": 1212,
"quantity": 1}' --header "Content-Type: application/json" -X POST http://blocmarket.herokuapp.com/createTrade

~~~~
        
## View order book (output to file)

~~~~
curl --header "Content-Type: application/json" -X POST http://blocmarket.herokuapp.com/viewOrderBook --output ob.txt
~~~~    
        
Also can load json from file with something like 
~~~~ 
curl -d "@data.json" -X POST http://blocmarket.herokuapp.com/data 
~~~~


# Matlab API examples


## Create user

~~~~
resp = webread('http://blocmarket.herokuapp.com/createUser')

~~~~

Returns something like

~~~~
'{"signingKey": '57f611e7cb7c5d4123be69663ccd06442c64db1a4f1161399dbb1b9bb6cc0e06'
           "traderId": '2'
      "verifyKey": 'ddaa5b0119a4ccfeffa36be0e6aac3682f798d021d5ded1d90512f2226e727b5'}'
~~~~

## Create market

~~~~
data = struct('signingKey', signingKey,...
'traderId', tId, ...
'verifyKey', verifyKey ,...
'marketRootId', 2, ...
'marketBranchId', 1, ...
'marketMin', 100, ...
'marketMax', 1e4)
resp = webwrite('http://blocmarket.herokuapp.com/createMarket', data, weboptions('RequestMethod', 'post','MediaType', 'application/json'))
~~~~

Returns something like

~~~~
'{
             "checks": 'True'
     "marketBranchId": 1
          "marketMax": 10000
          "marketMin": 100
       "marketRootId": 2
           "traderId": 1
}'
~~~~
## Create trade

~~~~
data = struct('signingKey', signingKey,...
'traderId', tId,...
'verifyKey', verifyKey ,...
'marketId', 1,...
'price', 5100,...
'quantity', 1)
resp = webwrite('http://blocmarket.herokuapp.com/createTrade', data, weboptions('MediaType', 'application/json'))
~~~~

Returns something like
~~~~
'{
       "checks": 'True'
     "marketId": 1
        "price": 5100
     "quantity": 1
     "traderId": 1
 '}S
~~~~

## View market bounds
~~~~
data = struct('marketId', 1)
resp = webwrite('http://blocmarket.herokuapp.com/viewMarketBounds', data, weboptions('RequestMethod', 'post','contentType', 'json', 'Timeout', 10))
~~~~

## View order book
~~~~
data = struct('marketId':1)
resp = webwrite('http://blocmarket.herokuapp.com/viewOrderBook', data, weboptions('RequestMethod', 'post','contentType', 'json'), 'Timeout', 10)
~~~~

## View trade summary

~~~~
data = struct('traderId',1);
resp = webwrite('http://blocmarket.herokuapp.com/viewTradeSummary', data, weboptions('RequestMethod', 'post','MediaType', 'application/json', 'Timeout', 10))
~~~~


# Powershell

TODO


    

test cases with mismatched traderId/verifyKey
views to specific market
javascript/powershell for api
enforce max decimals DONE
table based checkcollateral
put up on digitalocean

write up another layer of trading interface